## Тестирование Sparse Bayesian Regression

In [4]:
import torch
import torch.nn as nn
from bmm_multitask.sparse_bayesian_regression import SparseBayesianRegression

In [2]:
# Генерация синтетических данных
n_samples = 200
n_features = 10
X = torch.randn(n_samples, n_features)
true_w = torch.zeros(n_features)
true_w[:3] = torch.tensor([2.0, -3.0, 1.5])  # только первые 3 признака значимы
y = X @ true_w + 0.5 * torch.randn(n_samples)
Y = y.unsqueeze(1)

In [3]:
# Определение простой линейной модели
torch.manual_seed(0)
model = nn.Linear(n_features, 1, bias=False)

In [4]:
# Группы признаков (например, по 2 признака в группе)
group_indices = [list(range(i, i+2)) for i in range(0, n_features, 2)]

In [8]:
# Инициализация и обучение Sparse Bayesian Regression
sbr = SparseBayesianRegression(model, group_indices, device='cpu')
sbr.fit(X, Y, num_iter=20)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x1 and 2x1)

In [ ]:
# Предсказания и сравнение с обычной линейной регрессией
with torch.no_grad():
    y_pred_sbr = sbr.predict(X).squeeze()
    # Обычная линейная регрессия для сравнения
    ols = nn.Linear(n_features, 1, bias=False)
    optimizer = torch.optim.SGD(ols.parameters(), lr=0.1)
    for _ in range(200):
        optimizer.zero_grad()
        loss = ((ols(X).squeeze() - y) ** 2).mean()
        loss.backward()
        optimizer.step()
    y_pred_ols = ols(X).squeeze()

In [ ]:
# Визуализация весов
plt.figure(figsize=(10,4))
plt.stem(true_w.numpy(), linefmt='g-', markerfmt='go', basefmt=' ', label='Истинные веса')
plt.stem(sbr._get_flat_params().cpu().numpy(), linefmt='b-', markerfmt='bo', basefmt=' ', label='Sparse Bayesian')
plt.stem(ols.weight.detach().cpu().numpy().flatten(), linefmt='r--', markerfmt='ro', basefmt=' ', label='OLS')
plt.legend()
plt.title('Сравнение весов моделей')
plt.xlabel('Индекс признака')
plt.ylabel('Вес')
plt.show()

In [ ]:
# Визуализация предсказаний
plt.figure(figsize=(6,6))
plt.scatter(y.numpy(), y_pred_sbr.numpy(), alpha=0.7, label='Sparse Bayesian')
plt.scatter(y.numpy(), y_pred_ols.numpy(), alpha=0.7, label='OLS', marker='x')
plt.plot(y.numpy(), y.numpy(), 'k--', label='y = y')
plt.xlabel('Истинные значения')
plt.ylabel('Предсказанные значения')
plt.legend()
plt.title('Сравнение предсказаний моделей')
plt.show()